In [37]:
import requests
import csv
import time

# ═══════════════════════════════════════════════════════════════════════════════
# SCRIPT DE EXTRACCIÓN DE PRODUCTOS DE MERCADONA
# ═══════════════════════════════════════════════════════════════════════════════

# --- CONFIGURACIÓN ---
BASE_URL = "https://tienda.mercadona.es/api/"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36",
    "Accept": "application/json"
}
CSV_FILE = "productos_mercadona.csv"
DELAY_BETWEEN_REQUESTS = 0.3  # segundos

# --- VARIABLES GLOBALES ---
productos_guardados = set()  # IDs de productos únicos
total_productos = 0

# ═══════════════════════════════════════════════════════════════════════════════
# FUNCIONES DE API
# ═══════════════════════════════════════════════════════════════════════════════

def obtener_datos(url):
    """
    Hace una petición GET a la URL especificada.
    Retorna el JSON o None si hay error.
    """
    try:
        time.sleep(DELAY_BETWEEN_REQUESTS)
        respuesta = requests.get(url, headers=HEADERS, timeout=10)
        respuesta.raise_for_status()
        return respuesta.json()
    except requests.exceptions.RequestException as error:
        print(f"⚠️  Error al obtener {url}: {error}")
        return None


def obtener_categorias_principales():
    """
    Obtiene todas las categorías principales de Mercadona.
    Retorna una lista de categorías.
    """
    url = f"{BASE_URL}categories/"
    datos = obtener_datos(url)
    if datos and "results" in datos:
        return datos["results"]
    return []


def procesar_categoria(categoria_id, ruta_categorias, csv_writer):
    """
    Procesa una categoría de forma recursiva.
    
    Estructura de la API:
    - GET /categories/{id} retorna un objeto con un array "categories"
    - Cada elemento de "categories" es una subcategoría que puede tener "products"
    - Los productos están en: subcategoria["products"]
    
    Args:
        categoria_id: ID de la categoría a procesar
        ruta_categorias: Lista con la ruta de categorías padre
        csv_writer: Writer del CSV para guardar productos
    """
    global total_productos
    
    url = f"{BASE_URL}categories/{categoria_id}"
    datos = obtener_datos(url)
    
    if not datos:
        return
    
    # Verificar si tiene subcategorías
    if "categories" not in datos:
        return
    
    subcategorias = datos["categories"]
    if not subcategorias:
        return
    
    # Procesar cada subcategoría
    for subcategoria in subcategorias:
        subcat_id = subcategoria.get("id")
        subcat_nombre = subcategoria.get("name")
        
        if not subcat_id or not subcat_nombre:
            continue
        
        # Construir ruta completa de la categoría
        ruta_completa = ruta_categorias + [subcat_nombre]
        ruta_str = " > ".join(ruta_completa)
        
        print(f"\n📁 {ruta_str} (ID: {subcat_id})")
        
        # IMPORTANTE: Extraer productos DIRECTAMENTE de esta subcategoría
        if "products" in subcategoria:
            productos = subcategoria["products"]
            
            if productos:  # Solo si hay productos
                print(f"   ✅ {len(productos)} productos encontrados")
                
                # Procesar cada producto
                for producto in productos:
                    guardar_producto(producto, ruta_str, csv_writer)
                    total_productos += 1
            else:
                print(f"   ℹ️  Sin productos")
        else:
            print(f"   ℹ️  Sin productos")
        
        # Llamada recursiva para procesar subcategorías anidadas dentro de esta
        procesar_categoria(subcat_id, ruta_completa, csv_writer)


def guardar_producto(producto, ruta_categoria, csv_writer):
    """
    Guarda un producto en el CSV si no ha sido guardado antes.
    También lo imprime en consola.
    
    Args:
        producto: Diccionario con los datos del producto
        ruta_categoria: String con la ruta completa de categorías
        csv_writer: Writer del CSV
    """
    producto_id = producto.get("id")
    
    # Evitar duplicados
    if not producto_id or producto_id in productos_guardados:
        return
    
    # Extraer datos del producto
    nombre = producto.get("display_name", "")
    packaging = producto.get("packaging", "")
    
    # Extraer información de precios
    price_info = producto.get("price_instructions", {})
    precio_unidad = price_info.get("unit_price", "")
    precio_bulk = price_info.get("bulk_price", "")
    precio_referencia = price_info.get("reference_price", "")
    formato_referencia = price_info.get("reference_format", "")
    
    # Construir string de precio de referencia
    precio_ref_completo = f"{precio_referencia} {formato_referencia}".strip()
    
    # Imprimir en consola (como en el PHP)
    print(f"      • ID {producto_id}: {nombre} ({precio_unidad}€)")
    
    # Escribir en CSV
    csv_writer.writerow({
        "id": producto_id,
        "nombre": nombre,
        "subtitulo": packaging,
        "precio_unidad": precio_unidad,
        "precio_unidad_por": precio_bulk,
        "precio_referencia": precio_ref_completo,
        "ruta_categoria": ruta_categoria
    })
    
    # Marcar como guardado
    productos_guardados.add(producto_id)


# ═══════════════════════════════════════════════════════════════════════════════
# EJECUCIÓN PRINCIPAL
# ═══════════════════════════════════════════════════════════════════════════════

def main():
    """Función principal que ejecuta el proceso completo de extracción."""
    
    print("╔" + "═" * 78 + "╗")
    print("║" + " " * 20 + "EXTRACTOR DE PRODUCTOS MERCADONA" + " " * 26 + "║")
    print("╚" + "═" * 78 + "╝")
    print()
    
    # Definir columnas del CSV
    columnas = [
        "id",
        "nombre",
        "subtitulo",
        "precio_unidad",
        "precio_unidad_por",
        "precio_referencia",
        "ruta_categoria"
    ]
    
    try:
        # Abrir archivo CSV
        with open(CSV_FILE, 'w', newline='', encoding='utf-8') as archivo_csv:
            writer = csv.DictWriter(archivo_csv, fieldnames=columnas, delimiter=';')
            writer.writeheader()
            
            # Obtener categorías principales
            print("🔍 Obteniendo categorías principales...\n")
            categorias = obtener_categorias_principales()
            
            if not categorias:
                print("❌ No se pudieron obtener las categorías")
                return
            
            print(f"✅ {len(categorias)} categorías principales encontradas\n")
            
            # Procesar cada categoría principal
            for categoria in categorias:
                cat_id = categoria.get("id")
                cat_nombre = categoria.get("name")
                
                if not cat_id or not cat_nombre:
                    continue
                
                print("\n" + "═" * 80)
                print(f"🏷️  {cat_nombre.upper()} (ID: {cat_id})")
                print("═" * 80)
                
                # Procesar esta categoría y todas sus subcategorías
                procesar_categoria(cat_id, [cat_nombre], writer)
            
            # Resumen final
            print("\n" + "╔" + "═" * 78 + "╗")
            print("║" + " " * 30 + "PROCESO COMPLETADO" + " " * 30 + "║")
            print("╠" + "═" * 78 + "╣")
            print(f"║  📊 Total de productos encontrados: {total_productos:<44} ║")
            print(f"║  💾 Productos únicos guardados:     {len(productos_guardados):<44} ║")
            print(f"║  📄 Archivo generado:                {CSV_FILE:<44} ║")
            print("╚" + "═" * 78 + "╝")
            
    except Exception as error:
        print(f"\n❌ Error inesperado: {error}")
        import traceback
        traceback.print_exc()


# Ejecutar el script
if __name__ == "__main__":
    main()


╔══════════════════════════════════════════════════════════════════════════════╗
║                    EXTRACTOR DE PRODUCTOS MERCADONA                          ║
╚══════════════════════════════════════════════════════════════════════════════╝

🔍 Obteniendo categorías principales...

✅ 26 categorías principales encontradas


════════════════════════════════════════════════════════════════════════════════
🏷️  ACEITE, ESPECIAS Y SALSAS (ID: 12)
════════════════════════════════════════════════════════════════════════════════
⚠️  Error al obtener https://tienda.mercadona.es/api/categories/12: 404 Client Error: Not Found for url: https://tienda.mercadona.es/api/categories/12

════════════════════════════════════════════════════════════════════════════════
🏷️  AGUA Y REFRESCOS (ID: 18)
════════════════════════════════════════════════════════════════════════════════
⚠️  Error al obtener https://tienda.mercadona.es/api/categories/18: 404 Client Error: Not Found for url: https://tienda.mercadona

In [50]:
import requests
import json

# PRUEBA CON CATEGORÍA 45 - DEBUG DETALLADO

url = "https://tienda.mercadona.es/api/categories/42"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36",
    "Accept": "application/json"
}

print("🔍 Haciendo petición a:", url)
print("=" * 80)

response = requests.get(url, headers=headers, timeout=10)
print(f"✅ Status Code: {response.status_code}")

data = response.json()

print(f"\n📦 Claves principales en la respuesta:")
print(f"   {list(data.keys())}")

print(f"\n📋 Información de la categoría:")
print(f"   ID: {data.get('id')}")
print(f"   Nombre: {data.get('name')}")

if "categories" in data:
    subcategorias = data["categories"]
    print(f"\n✅ Tiene {len(subcategorias)} subcategoría(s)")
    
    for i, subcat in enumerate(subcategorias):
        print(f"\n   {'─' * 70}")
        print(f"   📁 Subcategoría {i+1}:")
        print(f"      ID: {subcat.get('id')}")
        print(f"      Nombre: {subcat.get('name')}")
        print(f"      Claves: {list(subcat.keys())}")
        
        if "products" in subcat:
            products = subcat["products"]
            print(f"      ✅ PRODUCTOS ENCONTRADOS: {len(products)}")
            
            # Mostrar los primeros 3 productos
            for j, product in enumerate(products[:3]):
                prod_id = product.get("id")
                prod_name = product.get("display_name")
                price_info = product.get("price_instructions", {})
                price = price_info.get("unit_price", "N/A")
                
                print(f"         {j+1}. ID {prod_id}: {prod_name} ({price}€)")
            
            if len(products) > 3:
                print(f"         ... y {len(products) - 3} productos más")
        else:
            print(f"      ⚠️ NO tiene clave 'products'")
            
        # Verificar si tiene subcategorías anidadas
        if "categories" in subcat:
            print(f"      🔄 Tiene subcategorías anidadas: {len(subcat['categories'])}")
else:
    print("\n❌ NO tiene clave 'categories' en la respuesta principal")

print("\n" + "=" * 80)
print("✅ DEBUG COMPLETADO")


🔍 Haciendo petición a: https://tienda.mercadona.es/api/categories/42
✅ Status Code: 200

📦 Claves principales en la respuesta:
   ['id', 'name', 'order', 'layout', 'published', 'is_extended', 'categories']

📋 Información de la categoría:
   ID: 42
   Nombre: Conejo y cordero

✅ Tiene 2 subcategoría(s)

   ──────────────────────────────────────────────────────────────────────
   📁 Subcategoría 1:
      ID: 285
      Nombre: Conejo
      Claves: ['id', 'name', 'order', 'layout', 'published', 'is_extended', 'image', 'subtitle', 'products']
      ✅ PRODUCTOS ENCONTRADOS: 3
         1. ID 25972: Medio conejo troceado (5.52€)
         2. ID 2797: Conejo entero (10.16€)
         3. ID 25191: Paletillas de conejo (7.96€)

   ──────────────────────────────────────────────────────────────────────
   📁 Subcategoría 2:
      ID: 284
      Nombre: Cordero
      Claves: ['id', 'name', 'order', 'layout', 'published', 'is_extended', 'image', 'subtitle', 'products']
      ✅ PRODUCTOS ENCONTRADOS: 5
   

In [41]:
import requests
import json
import time

# CONSTRUIR DICCIONARIO DE CATEGORÍAS Y SUBCATEGORÍAS
BASE_URL = "https://tienda.mercadona.es/api/"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36",
    "Accept": "application/json"
}

print("╔" + "═" * 78 + "╗")
print("║" + " " * 18 + "CREANDO DICCIONARIO DE CATEGORÍAS MERCADONA" + " " * 17 + "║")
print("╚" + "═" * 78 + "╝\n")

# Diccionario que almacenará nombre -> ID
categorias_dict = {}

# También crear un diccionario con estructura jerárquica
categorias_jerarquia = {}

def normalizar_nombre(nombre):
    """Normaliza el nombre para búsqueda (minúsculas, sin tildes)"""
    import unicodedata
    nombre = nombre.lower().strip()
    # Eliminar tildes
    nombre = ''.join(
        c for c in unicodedata.normalize('NFD', nombre)
        if unicodedata.category(c) != 'Mn'
    )
    return nombre

print("🔍 Obteniendo categorías principales...\n")
categorias_url = f"{BASE_URL}categories/"
response = requests.get(categorias_url, headers=headers, timeout=10)

if response.status_code != 200:
    print(f"❌ Error al obtener categorías: {response.status_code}")
else:
    data = response.json()
    categorias_principales = data.get("results", [])
    print(f"✅ {len(categorias_principales)} categorías principales encontradas\n")
    
    total_subcategorias = 0
    
    # Recorrer cada categoría principal
    for idx, categoria in enumerate(categorias_principales):
        cat_id = categoria.get("id")
        cat_nombre = categoria.get("name")
        
        if not cat_id or not cat_nombre:
            continue
        
        print(f"📦 [{idx+1}/{len(categorias_principales)}] Procesando: {cat_nombre}")
        
        # Guardar categoría principal
        cat_nombre_norm = normalizar_nombre(cat_nombre)
        categorias_dict[cat_nombre_norm] = cat_id
        categorias_dict[cat_nombre] = cat_id
        
        # Inicializar jerarquía
        categorias_jerarquia[cat_nombre] = {
            "id": cat_id,
            "nombre_normalizado": cat_nombre_norm,
            "subcategorias": {}
        }
        
        # Obtener subcategorías
        url = f"{BASE_URL}categories/{cat_id}"
        
        try:
            time.sleep(0.3)
            cat_response = requests.get(url, headers=headers, timeout=10)
            
            if cat_response.status_code != 200:
                print(f"   ⚠️  Error: {cat_response.status_code}")
                continue
            
            cat_data = cat_response.json()
            
            if "categories" in cat_data:
                subcategorias = cat_data["categories"]
                print(f"   ✅ {len(subcategorias)} subcategorías")
                total_subcategorias += len(subcategorias)
                
                for subcat in subcategorias:
                    subcat_id = subcat.get("id")
                    subcat_nombre = subcat.get("name")
                    
                    if not subcat_id or not subcat_nombre:
                        continue
                    
                    subcat_nombre_norm = normalizar_nombre(subcat_nombre)
                    categorias_dict[subcat_nombre_norm] = subcat_id
                    categorias_dict[subcat_nombre] = subcat_id
                    
                    categorias_jerarquia[cat_nombre]["subcategorias"][subcat_nombre] = {
                        "id": subcat_id,
                        "nombre_normalizado": subcat_nombre_norm,
                        "num_productos": len(subcat.get("products", []))
                    }
            else:
                print(f"   ℹ️  Sin subcategorías")
                
        except Exception as e:
            print(f"   ❌ Error: {e}")
    
    print("\n" + "╔" + "═" * 78 + "╗")
    print("║" + " " * 30 + "DICCIONARIO CREADO" + " " * 28 + "║")
    print("╠" + "═" * 78 + "╣")
    print(f"║  📊 Categorías principales:          {len(categorias_principales):<38} ║")
    print(f"║  📁 Total de subcategorías:          {total_subcategorias:<38} ║")
    print(f"║  📝 Total de entradas en diccionario: {len(categorias_dict):<38} ║")
    print("╚" + "═" * 78 + "╝")
    
    def buscar_categoria_por_nombre(nombre_busqueda):
        """Busca una categoría por su nombre y retorna su ID"""
        if nombre_busqueda in categorias_dict:
            return categorias_dict[nombre_busqueda]
        
        nombre_norm = normalizar_nombre(nombre_busqueda)
        if nombre_norm in categorias_dict:
            return categorias_dict[nombre_norm]
        
        for nombre, cat_id in categorias_dict.items():
            if nombre_norm in normalizar_nombre(nombre):
                return cat_id
        
        return None
    
    print("\n" + "=" * 80)
    print("📚 EJEMPLOS DE USO:")
    print("=" * 80)
    
    ejemplos = ["Carne", "Mortadela", "Aceite", "Pan", "Yogures"]
    
    for ejemplo in ejemplos:
        cat_id = buscar_categoria_por_nombre(ejemplo)
        if cat_id:
            print(f"✅ '{ejemplo}' → ID: {cat_id}")
        else:
            print(f"❌ '{ejemplo}' → No encontrado")
    
    print("\n💡 Uso: categorias_dict['nombre'] o buscar_categoria_por_nombre('nombre')")



╔══════════════════════════════════════════════════════════════════════════════╗
║                  CREANDO DICCIONARIO DE CATEGORÍAS MERCADONA                 ║
╚══════════════════════════════════════════════════════════════════════════════╝

🔍 Obteniendo categorías principales...

✅ 26 categorías principales encontradas

📦 [1/26] Procesando: Aceite, especias y salsas
   ⚠️  Error: 404
📦 [2/26] Procesando: Agua y refrescos
   ⚠️  Error: 404
📦 [3/26] Procesando: Aperitivos
   ⚠️  Error: 404
📦 [4/26] Procesando: Arroz, legumbres y pasta
   ⚠️  Error: 404
📦 [5/26] Procesando: Azúcar, caramelos y chocolate
   ⚠️  Error: 404
📦 [6/26] Procesando: Bebé
   ⚠️  Error: 404
📦 [7/26] Procesando: Bodega
   ⚠️  Error: 404
📦 [8/26] Procesando: Cacao, café e infusiones
   ⚠️  Error: 404
📦 [9/26] Procesando: Carne
   ⚠️  Error: 404
📦 [10/26] Procesando: Cereales y galletas
   ⚠️  Error: 404
📦 [11/26] Procesando: Charcutería y quesos
   ⚠️  Error: 404
📦 [12/26] Procesando: Congelados
   ⚠️  Error: 404


In [46]:
# ═══════════════════════════════════════════════════════════════════════════════
# FUNCIÓN PARA BUSCAR CATEGORÍAS POR NOMBRE DE PRODUCTO
# ═══════════════════════════════════════════════════════════════════════════════

def buscar_categorias_por_producto(nombre_producto):
    """
    Busca todas las categorías que contengan el nombre del producto en su nombre.
    
    Args:
        nombre_producto: Nombre del producto a buscar (ej: "pollo", "leche", "pan")
        
    Returns:
        Lista de diccionarios con información de las categorías encontradas:
        [
            {
                "nombre": "Nombre de la categoría",
                "id": 123,
                "nombre_normalizado": "nombre normalizado"
            },
            ...
        ]
    """
    if not nombre_producto:
        return []
    
    # Normalizar el nombre del producto buscado
    producto_norm = normalizar_nombre(nombre_producto)
    
    resultados = []
    categorias_encontradas = set()  # Para evitar duplicados
    
    # Buscar en categorias_dict
    for nombre_cat, cat_id in categorias_dict.items():
        nombre_cat_norm = normalizar_nombre(str(nombre_cat))
        
        # Verificar si el producto está contenido en el nombre de la categoría
        if producto_norm in nombre_cat_norm:
            # Evitar duplicados por tener nombre original y normalizado
            if cat_id not in categorias_encontradas:
                resultados.append({
                    "nombre": nombre_cat,
                    "id": cat_id,
                    "nombre_normalizado": nombre_cat_norm
                })
                categorias_encontradas.add(cat_id)
    
    return resultados


def buscar_ids_categorias_por_producto(nombre_producto):
    """
    Versión simplificada que solo devuelve los IDs de las categorías.
    
    Args:
        nombre_producto: Nombre del producto a buscar
        
    Returns:
        Lista de IDs de categorías encontradas [123, 456, ...]
    """
    resultados = buscar_categorias_por_producto(nombre_producto)
    return [r["id"] for r in resultados]


# ═══════════════════════════════════════════════════════════════════════════════
# PRUEBAS Y EJEMPLOS DE USO
# ═══════════════════════════════════════════════════════════════════════════════

print("╔" + "═" * 78 + "╗")
print("║" + " " * 22 + "BÚSQUEDA DE CATEGORÍAS POR PRODUCTO" + " " * 21 + "║")
print("╚" + "═" * 78 + "╝\n")

# Lista de productos para probar
productos_prueba = [
    "pollo",
    "leche",
    "pan",
    "queso",
    "yogur",
    "carne",
    "pescado",
    "aceite",
    "chocolate",
    "cafe"
]

print("🔍 Buscando categorías para diferentes productos...\n")

for producto in productos_prueba:
    print("─" * 80)
    print(f"🔎 Buscando: '{producto}'")
    print("─" * 80)
    
    # Buscar con la función completa
    resultados = buscar_categorias_por_producto(producto)
    
    if resultados:
        print(f"✅ {len(resultados)} categoría(s) encontrada(s):\n")
        for i, resultado in enumerate(resultados, 1):
            print(f"   {i}. {resultado['nombre']}")
            print(f"      ID: {resultado['id']}")
            print(f"      Normalizado: {resultado['nombre_normalizado']}")
            print()
        
        # Mostrar también solo los IDs
        ids = buscar_ids_categorias_por_producto(producto)
        print(f"   📋 IDs: {ids}")
    else:
        print(f"❌ No se encontraron categorías para '{producto}'")
    
    print()

print("\n" + "╔" + "═" * 78 + "╗")
print("║" + " " * 30 + "EJEMPLOS DE USO" + " " * 32 + "║")
print("╠" + "═" * 78 + "╣")
print("║  1. buscar_categorias_por_producto('pollo')                                  ║")
print("║     → Devuelve lista completa con nombre, ID y normalizado                  ║")
print("║                                                                              ║")
print("║  2. buscar_ids_categorias_por_producto('pollo')                              ║")
print("║     → Devuelve solo los IDs [123, 456, ...]                                 ║")
print("╚" + "═" * 78 + "╝")



╔══════════════════════════════════════════════════════════════════════════════╗
║                      BÚSQUEDA DE CATEGORÍAS POR PRODUCTO                     ║
╚══════════════════════════════════════════════════════════════════════════════╝

🔍 Buscando categorías para diferentes productos...

────────────────────────────────────────────────────────────────────────────────
🔎 Buscando: 'pollo'
────────────────────────────────────────────────────────────────────────────────
❌ No se encontraron categorías para 'pollo'

────────────────────────────────────────────────────────────────────────────────
🔎 Buscando: 'leche'
────────────────────────────────────────────────────────────────────────────────
✅ 1 categoría(s) encontrada(s):

   1. huevos, leche y mantequilla
      ID: 6
      Normalizado: huevos, leche y mantequilla

   📋 IDs: [6]

────────────────────────────────────────────────────────────────────────────────
🔎 Buscando: 'pan'
──────────────────────────────────────────────────────

In [44]:
print(categorias_principales)

[{'id': 12, 'name': 'Aceite, especias y salsas', 'order': 7, 'is_extended': False, 'categories': [{'id': 112, 'name': 'Aceite, vinagre y sal', 'order': 7, 'layout': 1, 'published': True, 'is_extended': False}, {'id': 115, 'name': 'Especias', 'order': 7, 'layout': 1, 'published': True, 'is_extended': False}, {'id': 116, 'name': 'Mayonesa, ketchup y mostaza', 'order': 7, 'layout': 1, 'published': True, 'is_extended': False}, {'id': 117, 'name': 'Otras salsas', 'order': 7, 'layout': 1, 'published': True, 'is_extended': False}]}, {'id': 18, 'name': 'Agua y refrescos', 'order': 8, 'is_extended': False, 'categories': [{'id': 156, 'name': 'Agua', 'order': 8, 'layout': 1, 'published': True, 'is_extended': False}, {'id': 163, 'name': 'Isotónico y energético', 'order': 8, 'layout': 1, 'published': True, 'is_extended': False}, {'id': 158, 'name': 'Refresco de cola', 'order': 8, 'layout': 1, 'published': True, 'is_extended': False}, {'id': 159, 'name': 'Refresco de naranja y de limón', 'order': 8,